In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

: 

In [ ]:
class GRN(nn.Module):
    def __init__(self, d_in, d_hidden, d_out):
        super().__init__()
        self.fc1 = nn.Linear(d_in, d_hidden)
        self.fc2 = nn.Linear(d_hidden, d_out)
        self.gate = nn.Linear(d_out, d_out)
        self.skip = nn.Linear(d_in, d_out) if d_in != d_out else nn.Identity()
        self.norm = nn.LayerNorm(d_out)

    def forward(self, x):
        h = F.elu(self.fc1(x))
        h = self.fc2(h)
        g = torch.sigmoid(self.gate(h))
        return self.norm(g * h + (1 - g) * self.skip(x))


In [ ]:
class StaticEncoder(nn.Module):
    def __init__(self, d_static, d_model):
        super().__init__()
        self.grn = GRN(d_static, d_model, d_model)

    def forward(self, s):
        return self.grn(s)


In [ ]:
class VariableSelectionNetwork(nn.Module):
    def __init__(self, num_vars, d_model):
        super().__init__()
        self.var_grns = nn.ModuleList([
            GRN(1, d_model, d_model) for _ in range(num_vars)
        ])
        self.weight_grn = GRN(num_vars, d_model, num_vars)

    def forward(self, x):
        # x: [B, T, num_vars]
        var_embeds = []
        for i, grn in enumerate(self.var_grns):
            var_embeds.append(grn(x[..., i:i+1]))
        var_embeds = torch.stack(var_embeds, dim=-2)  # [B,T,num_vars,d]

        weights = self.weight_grn(x).softmax(dim=-1)  # [B,T,num_vars]
        fused = (weights.unsqueeze(-1) * var_embeds).sum(dim=-2)
        return fused, weights


In [ ]:
class ContextEnrichment(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.grn = GRN(d_model * 2, d_model, d_model)

    def forward(self, temporal, context):
        context = context.unsqueeze(1).expand_as(temporal)
        return self.grn(torch.cat([temporal, context], dim=-1))


In [ ]:
class TemporalAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=num_heads,
            batch_first=True
        )
        self.grn = GRN(d_model, d_model, d_model)

    def forward(self, x):
        T = x.size(1)
        mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(x.device)
        attn_out, attn_weights = self.attn(x, x, x, attn_mask=mask)
        out = self.grn(attn_out + x)
        return out, attn_weights


In [ ]:
class PredictionHead(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        return self.fc(x[:, -1])


In [ ]:
class MiniTFT(nn.Module):
    def __init__(self, n_obs, n_known, d_static, d_model=32):
        super().__init__()
        self.static_enc = StaticEncoder(d_static, d_model)

        self.obs_vsn = VariableSelectionNetwork(n_obs, d_model)
        self.known_vsn = VariableSelectionNetwork(n_known, d_model)

        self.enrich = ContextEnrichment(d_model)
        self.attn = TemporalAttention(d_model, num_heads=4)
        
        self.post_attn_grn = GRN(d_model, d_model, d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        
        self.head = PredictionHead(d_model)

    def forward(self, obs, known, static):
        # Preparation for 3 types of inputs
        s = self.static_enc(static)
        obs_fused,_ = self.obs_vsn(obs)
        known_fused,_ = self.known_vsn(known)
        # Locality enhanchement
        x = obs_fused + known_fused
        # Temporal processing
        x = self.enrich(x, s)
        # Temporal attention
        attn_out,attn_weights = self.attn(x)
        x = self.layer_norm(attn_out + x)
        x = self.layer_norm(self.post_attn_grn(x) + x)
        return self.head(x), attn_weights


In [ ]:

df = pd.read_parquet("../../data/features/BTC_features.parquet")
print(df.shape)
print(df.info())
print(df.head(1))
print("---------------------------------------")
print(df.columns.tolist())


In [ ]:
static_cols = ["symbol", "source"]
observed_cols = [
    "open", "high", "low", "close", "volume",
    "ema_34", "ema_89", "ema_200",
    "rsi_14", "macd", "log_return", "vol_20"
] + [c for c in df.columns if "lag_" in c]
known_cols = [
    "building_permits", "consumer_confidence", "cpi",
    "fed_funds_rate", "gdp", "industrial_production",
    "money_supply_m1", "money_supply_m2",
    "nonfarm_payrolls", "pce_inflation",
    "ppi", "retail_sales", "trade_balance",
    "unemployment_rate"
] + [c for c in df.columns if c.startswith("fed_emb_")]
target_col = "next_close"

# train val test splitting
n = len(df)
train_end = int(0.70 * n)
val_end = int(0.85 * n)

df_train = df.iloc[:train_end]
df_val   = df.iloc[train_end:val_end]
df_test  = df.iloc[val_end:]


X_train = df_train[observed_cols + known_cols]
X_val   = df_val[observed_cols + known_cols]
X_test  = df_test[observed_cols + known_cols]

y_train = df_train[target_col].values
y_val   = df_val[target_col].values
y_test  = df_test[target_col].values
# Sliding window
LOOKBACK = 89

def build_windows(X, y, lookback):
    X_out, y_out = [], []
    for i in range(lookback, len(X)):
        X_out.append(X[i-lookback:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)

Xtr, ytr = build_windows(X_train, y_train, LOOKBACK)
Xva, yva = build_windows(X_val, y_val, LOOKBACK)
Xte, yte = build_windows(X_test, y_test, LOOKBACK)

# split input feature tensors
n_obs   = len(observed_cols)
n_known = len(known_cols)
#Tensor
obs_tr   = Xtr[:, :, :n_obs]
known_tr = Xtr[:, :, n_obs:]

static_tr = np.zeros((len(Xtr), 2))  # placeholder encoding


In [ ]:
def count_parameters(model):
    # Total parameters (including those frozen/not being trained)
    total_params = sum(p.numel() for p in model.parameters())
    
    # Trainable parameters only
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    return total_params

# Usage:
model = MiniTFT(n_obs=n_obs,
    n_known=n_known,
    d_static=2,
    d_model=32)
count_parameters(model)

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MiniTFT(
    n_obs=n_obs,
    n_known=n_known,
    d_static=2,
    d_model=32
).to(device)

pred = model(
    torch.tensor(obs_tr).float().to(device),
    torch.tensor(known_tr).float().to(device),
    torch.tensor(static_tr).float().to(device)
)